**TASK 2**
1. Read the csv or excel file. 
2. Change the datatypes of the column and check the memory usage before and after the change in the data types.
3. Dump the data into the mysql database.
4. index the column after the data is inserted.

Note: Please make a note we are dumping the data everyday so first delete the indexing from the table and then insert the data.


In [1]:
import pandas as pd #pandas for data analysis
import sqlite3 #sqlite for writing sql queries
import logging

**1. Reading the csv or excel file.**

In [2]:
df = pd.read_csv('/content/used_bikes.csv')

In [3]:
df

,bike_name,price,city,kms_driven,owner,age,power,brand
0,TVS Star City Plus Dual Tone 110cc,35000,Ahmedabad,17654,First Owner,3,110,TVS
1,Royal Enfield Classic 350cc,119900,Delhi,11000,First Owner,4,350,Royal Enfield
2,Triumph Daytona 675R,600000,Delhi,110,First Owner,8,675,Triumph
3,TVS Apache RTR 180cc,65000,Bangalore,16329,First Owner,4,180,TVS
4,Yamaha FZ S V 2.0 150cc-Ltd. Edition,80000,Bangalore,10000,First Owner,3,150,Yamaha
...,...,...,...,...,...,...,...,...
144,Royal Enfield Standard 350cc,115000,Hyderabad,14900,First Owner,3,350,Royal Enfield
145,Honda CB Shine 125cc Disc,65000,Mumbai,2000,First Owner,3,125,Honda
146,Honda CB Unicorn ABS 150cc,99000,Mumbai,4000,First Owner,2,150,Honda
147,Yamaha YZF-R15 2.0 150cc,68500,Delhi,68500,Second Owner,7,150,Yamaha


**2. Changing the datatypes of the column and checking the memory usage before and after the change in the data types.**

In [4]:
#checking memory usage before changing the datatype of price from int32 to int64
df.memory_usage()

Index          128
bike_name     1192
price         1192
city          1192
kms_driven    1192
owner         1192
age           1192
power         1192
brand         1192
dtype: int64

In [5]:
#total sum of memory usage
df.memory_usage().sum()

9664

In [6]:
#changing datatype of Price from int64 to int32
df = df.astype("int32",errors='ignore')

In [7]:
#checking memory usage
df.memory_usage()

Index          128
bike_name     1192
price          596
city          1192
kms_driven     596
owner         1192
age            596
power          596
brand         1192
dtype: int64

In [8]:
#total sum of memory usage
df.memory_usage().sum()

7280

**Dropping the Index**

In [9]:
#droping indexing from the table
df.reset_index(drop=True, inplace=True)

In [10]:
#index is being dropped
print(df.head())

                              bike_name   price  ... power          brand
0    TVS Star City Plus Dual Tone 110cc   35000  ...   110            TVS
1           Royal Enfield Classic 350cc  119900  ...   350  Royal Enfield
2                  Triumph Daytona 675R  600000  ...   675        Triumph
3                  TVS Apache RTR 180cc   65000  ...   180            TVS
4  Yamaha FZ S V 2.0 150cc-Ltd. Edition   80000  ...   150         Yamaha

[5 rows x 8 columns]


**3. Dumping the data into the mysql database.**

In [11]:
#dumping the file in MySQL 
# defining function to convert pandas dataframe into MySQL
def pd_to_sqlDB(input_df: pd.DataFrame,table_name: str,db_name: str = 'default.db') -> None:
  
    # Settingup local logging
    logging.basicConfig(level=logging.INFO,format='%(asctime)s %(levelname)s: %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S')
    
    # Finding columns in the dataframe
    cols = input_df.columns
    cols_string = ','.join(cols)
    val_wildcard_string = ','.join(['?'] * len(cols))

    #Connecting to a DB file if it exists, else crete a new file
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    logging.info(f'SQL DB {db_name} created')

    #Creating Table
    sql_string = f"""CREATE TABLE {table_name} ({cols_string});"""
    cur.execute(sql_string)
    logging.info(f'SQL Table {table_name} created with {len(cols)} columns')

    #Uploading the dataframe
    rows_to_upload = input_df.to_dict(orient='split')['data']
    sql_string = f"""INSERT INTO {table_name} ({cols_string}) VALUES ({val_wildcard_string});"""    
    cur.executemany(sql_string, rows_to_upload)
    logging.info(f'{len(rows_to_upload)} rows uploaded to {table_name}')

    #Commiting the changes and closing the connection
    con.commit()
    con.close()
 


In [12]:
#data dumped to SQL database by converting dataframe to sqlite database
pd_to_sqlDB(df,table_name='used_bikes',db_name='data.db')

2022-02-05 09:25:09 INFO: SQL DB data.db created
2022-02-05 09:25:09 INFO: SQL Table used_bikes created with 8 columns
2022-02-05 09:25:09 INFO: 149 rows uploaded to used_bikes


**4.indexing the column after the data is inserted.**

In [13]:
#Function Execute an SQL query and return the results as a pandas dataframe 
#this function also inserts indexing and returns data in pandas dataframe

def sql_query_to_pd(sql_query_string: str, db_name: str ='default.db') -> pd.DataFrame:
    # connecting to the SQL DB
    con = sqlite3.connect(db_name)
    # Executing the SQL query
    cursor = con.execute(sql_query_string)
    # Fetching the data and column names
    result_data = cursor.fetchall()
    cols = [description[0] for description in cursor.description]
    # Closing the connection
    con.close()
    # Returning as a dataframe
    return pd.DataFrame(result_data, columns=cols)

In [14]:
sql_query_string = 'SELECT * FROM used_bikes'
result_df = sql_query_to_pd(sql_query_string, db_name='data.db')
result_df

,bike_name,price,city,kms_driven,owner,age,power,brand
0,TVS Star City Plus Dual Tone 110cc,35000,Ahmedabad,17654,First Owner,3,110,TVS
1,Royal Enfield Classic 350cc,119900,Delhi,11000,First Owner,4,350,Royal Enfield
2,Triumph Daytona 675R,600000,Delhi,110,First Owner,8,675,Triumph
3,TVS Apache RTR 180cc,65000,Bangalore,16329,First Owner,4,180,TVS
4,Yamaha FZ S V 2.0 150cc-Ltd. Edition,80000,Bangalore,10000,First Owner,3,150,Yamaha
...,...,...,...,...,...,...,...,...
144,Royal Enfield Standard 350cc,115000,Hyderabad,14900,First Owner,3,350,Royal Enfield
145,Honda CB Shine 125cc Disc,65000,Mumbai,2000,First Owner,3,125,Honda
146,Honda CB Unicorn ABS 150cc,99000,Mumbai,4000,First Owner,2,150,Honda
147,Yamaha YZF-R15 2.0 150cc,68500,Delhi,68500,Second Owner,7,150,Yamaha
